In [ ]:
# not needed

%%bash
python -m pip install tomotopy
python -m pip install pyLDAvis
python -m pip install seaborn

# unzip corpus
corpus_filename = 'ted-transcripts2018.zip' # change to the name of your dataset zip file
# Create a ZipFile Object and load sample.zip in it
with ZipFile(corpus_filename, 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall()

## Import the necessary Python packages

In [1]:
# various packages
from zipfile import ZipFile
import os.path
from os import path
import glob
import re
from pathlib import Path
from imp import reload
from collections import Counter, Iterable
import datetime

# topic modeling / nlp packages
import tomotopy as tp
from gensim import similarities
import gensim.corpora as corpora

# visualisation / exploration
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pyLDAvis
from IPython.display import IFrame, Markdown, display

# These last lines of code suppress deprecation warnings displaying in the notebook
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import nltk
nltk.download('stopwords')

warnings.filterwarnings("ignore", category=DeprecationWarning) 

from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

<ipython-input-1-e8a43e7e1203>:9: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  from collections import Counter, Iterable
C:\Anaconda3\lib\site-packages\sklearn\linear_model\_least_angle.py:34: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
C:\Anaconda3\lib\site-packages\sklearn\linear_model\_least_angle.py:164: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If

## Define functions

The following cells contain functions to load a corpus from a directory of text files and preprocess the corpus.

In [2]:
# documents_list, source_list

def load_data_from_dir(path):
    file_list = glob.glob(path + '/*.txt')
    print(file_list)

    # create document list:
    documents_list = []
    source_list = []
    for filename in file_list:
        with open(filename, 'r', encoding='utf8') as f:
            text = f.read()
            f.close()
            if len(text.split()) > 100: # only keep docs longer than 100 words
                documents_list.append(text)
                source_list.append(os.path.basename(filename))
    print(f"Total Number of Documents in '{path}':",len(documents_list))
    return documents_list, source_list

In [3]:
# texts

def preprocess_data(doc_set, extra_stopwords={}):
    # adapted from https://www.datacamp.com/community/tutorials/discovering-hidden-topics-python
    
    # replace all newlines or multiple sequences of spaces with a standard space
    doc_set = [re.sub(r'\s+', ' ', doc) for doc in doc_set]
    
    # initialize regex tokenizer
    tokenizer = RegexpTokenizer(r'\w+')
    
    # create English stop words list
    en_stop = set(stopwords.words('english'))
    
    # add any extra stopwords
    if (len(extra_stopwords) > 0):
        en_stop = en_stop.union(extra_stopwords)
    
    # list for tokenized documents in loop
    texts = []
    # loop through document list
    for i in doc_set:
        # clean and tokenize document string
        raw = i.lower()
        tokens = tokenizer.tokenize(raw)
        # remove stop words from tokens
        stopped_tokens = [i for i in tokens if not i in en_stop]
        # add tokens to list
        texts.append(stopped_tokens)
    return texts

## Load and pre-process the corpus
Load the corpus and preprocess with additional stop words (if required).

The name of the directory that contains the extracted corpus text files should be assigned to the 'corpus_dirname' variable below. Make sure the only text files in this directory are those belonging to the corpus.

In [4]:
corpus_dirname = 'transcripts2018'

# adjust the path below to wherever you have the transcripts2018 folder
documents_list, source_list = load_data_from_dir(corpus_dirname)

# You can add extra stopwords here, between the curly brackets, in addition to NLTK's stopword list
doc_clean = preprocess_data(documents_list, {'applause', 'laughter'})

source_data = []
for talk in source_list:
    match = re.search(r'\d{4}-\d{2}-\d{2}', talk)
    date = datetime.datetime.strptime(match.group(), '%Y-%m-%d').strftime('%Y-%m')
    source_data.append([date, talk])

['transcripts2018\\2018-01-06-elizabeth_cawein_we_built_this_city_on_a_bold_music_identity.txt', 'transcripts2018\\2018-01-24-nilay_kulkarni_a_life_saving_invention_that_prevents_human_stampedes.txt', 'transcripts2018\\2018-01-27-christine_porath_why_being_nice_to_your_coworkers_is_good_for_business.txt', 'transcripts2018\\2018-02-08-dawn_wacek_a_librarian_s_case_against_overdue_book_fines.txt', 'transcripts2018\\2018-02-17-david_lang_a_plan_to_protect_our_world_s_ocean.txt', 'transcripts2018\\2018-02-27-david_korins_a_revelation_of_space.txt', 'transcripts2018\\2018-03-03-jonathan_wilker_what_sticky_sea_creatures_can_teach_us_about_making_glue.txt', 'transcripts2018\\2018-03-03-juliet_brophy_how_a_new_species_of_ancestors_is_changing_our_theory_of_human_evolution.txt', 'transcripts2018\\2018-03-03-kriti_sharma_how_to_keep_human_biases_out_of_ai.txt', 'transcripts2018\\2018-03-10-matt_russo_what_does_the_universe_sound_like_a_musical_tour.txt', 'transcripts2018\\2018-03-15-caroline_wea

In [5]:
print(documents_list[0])

Each of these songs
represents a scene, a movement,
in some cases, a sonic revolution
that completely altered
the course of popular music.
They're all also calling cards,
almost, for those cities,
songs totally linked
with their city's identity,
and it might be why you probably
consider them to be music cities.
Now, the magical mythical thing,
the thing we kind of all love
about stories like these
is that those cities weren't doing
anything in particular
to make those moments happen.
There's no formula for capturing
lightning in a bottle.
A formula didn't give us grunge music
or introduce Tupac to Dr. Dre,
and there's definitely no blueprint
for how to open your record business
in a South Memphis neighborhood
that, turns out,
is home to Booker T. Jones,
William Bell and Albert King.

So this is just something
that happens, then, right?
When the stars perfectly align,
great music just happens.
And in the meantime,
New York and Nashville
can churn out the hits
that come through our radio

In [6]:
print(source_list)

['2018-01-06-elizabeth_cawein_we_built_this_city_on_a_bold_music_identity.txt', '2018-01-24-nilay_kulkarni_a_life_saving_invention_that_prevents_human_stampedes.txt', '2018-01-27-christine_porath_why_being_nice_to_your_coworkers_is_good_for_business.txt', '2018-02-08-dawn_wacek_a_librarian_s_case_against_overdue_book_fines.txt', '2018-02-17-david_lang_a_plan_to_protect_our_world_s_ocean.txt', '2018-02-27-david_korins_a_revelation_of_space.txt', '2018-03-03-jonathan_wilker_what_sticky_sea_creatures_can_teach_us_about_making_glue.txt', '2018-03-03-juliet_brophy_how_a_new_species_of_ancestors_is_changing_our_theory_of_human_evolution.txt', '2018-03-03-kriti_sharma_how_to_keep_human_biases_out_of_ai.txt', '2018-03-10-matt_russo_what_does_the_universe_sound_like_a_musical_tour.txt', '2018-03-15-caroline_weaver_why_the_pencil_is_perfect.txt', '2018-03-15-daniel_engber_how_the_progress_bar_keeps_you_sane.txt', '2018-03-15-david_rockwell_the_hidden_ways_stairs_shape_your_life.txt', '2018-03-15

In [7]:
print(doc_clean)

[['songs', 'represents', 'scene', 'movement', 'cases', 'sonic', 'revolution', 'completely', 'altered', 'course', 'popular', 'music', 'also', 'calling', 'cards', 'almost', 'cities', 'songs', 'totally', 'linked', 'city', 'identity', 'might', 'probably', 'consider', 'music', 'cities', 'magical', 'mythical', 'thing', 'thing', 'kind', 'love', 'stories', 'like', 'cities', 'anything', 'particular', 'make', 'moments', 'happen', 'formula', 'capturing', 'lightning', 'bottle', 'formula', 'give', 'us', 'grunge', 'music', 'introduce', 'tupac', 'dr', 'dre', 'definitely', 'blueprint', 'open', 'record', 'business', 'south', 'memphis', 'neighborhood', 'turns', 'home', 'booker', 'jones', 'william', 'bell', 'albert', 'king', 'something', 'happens', 'right', 'stars', 'perfectly', 'align', 'great', 'music', 'happens', 'meantime', 'new', 'york', 'nashville', 'churn', 'hits', 'come', 'radios', 'define', 'generations', 'soundtrack', 'weddings', 'funerals', 'everything', 'well', 'know', 'idea', 'deadly', 'bori

In [8]:
# Minimum frequency of words (integer)
# Words with a smaller document frequency than min_df are excluded from the model
# Default is 0 - i.e. no words are excluded
# For more info see https://bab2min.github.io/tomotopy/v0.12.3/en/#vocabulary-controlling-using-cf-and-df
min_doc_freq = 3

# Number of top words to be removed (integer)
# Setting this to 1 or more removes common words from the model
# Default is 0 - i.e. no words are excluded
remove_top = 5

# Number of topics to return, between 1 and 32767
num_topics = 15

# You can read more about the following alpha and beta hyperparameters here:
# https://medium.com/@lettier/how-does-lda-work-ill-explain-using-emoji-108abf40fa7d

# Alpha
# Hyperparameter of Dirichlet distribution for document-topic
# Controls the mixture of topics for any given document
# a float
doc_topic = 0.1

# Beta
# Hyperparameter of Dirichlet distribution for topic-word
# Note this is 'eta' in tomotopy - it's not a typo!
# Controls the distribution of words per topic
# a float
topic_word = 0.01

# Set the burn-in
# Burn-in is a number of initial iterations that are discarded
brn_in = 5

# Number of iterations of the Gibbs sampler
# If we specify 20 here, we will run 200 (10*20) iterations of Gibbs sampling total
# The higher the number, the longer the model training will take, but the more independent the samples will be
num_iterations = 100

# Set the top n words from the topic to display in the output of results
num_topic_words = 10

In [9]:
# Adapted from https://bab2min.github.io/tomotopy/v0.12.2/en/

# Intialize the model

# The default term weighting is used
model = tp.LDAModel(tw=tp.TermWeight.ONE,
                    min_df=min_doc_freq, 
                    rm_top=remove_top, 
                    k=num_topics, 
                    alpha=doc_topic, 
                    eta=topic_word
                   )

model.burn_in = brn_in

# Add each document to the model
for text in doc_clean:
    model.add_doc(text)

print("Topic Model Training...\n")

# train the model
# the loop reports LL/word every 10 iterations
# this is a measure of model fit to the data (higher is better)
for i in range(0, 100, 10):
    model.train(iter=num_iterations)

topics = []
topic_individual_words = []
for topic_number in range(0, num_topics):
    topic_words = ' '.join(word for word, prob in model.get_topic_words(topic_id=topic_number, top_n=num_topic_words))
    print(f'\nTop 10 words of topic #{topic_number}\n')
    print(model.get_topic_words(topic_id=topic_number, top_n=num_topic_words))
    print()
    topics.append(topic_words)
    topic_individual_words.append(topic_words.split())
    
    
print("\nModel Summary\n")
model.summary()

Topic Model Training...


Top 10 words of topic #0

[('get', 0.01880430243909359), ('think', 0.018198417499661446), ('know', 0.018003668636083603), ('going', 0.017873836681246758), ('really', 0.0175925325602293), ('make', 0.016229290515184402), ('way', 0.014195245690643787), ('right', 0.01361099909991026), ('want', 0.013459527865052223), ('things', 0.013199863024055958)]


Top 10 words of topic #1

[('also', 0.013874180614948273), ('could', 0.012028616853058338), ('around', 0.011646776460111141), ('world', 0.011392216198146343), ('new', 0.010564894415438175), ('different', 0.010119413956999779), ('time', 0.009673932567238808), ('building', 0.00948301237076521), ('design', 0.008273850195109844), ('across', 0.00808293092995882)]


Top 10 words of topic #2

[('internet', 0.022472308948636055), ('ca', 0.0183111559599638), ('online', 0.016022520139813423), ('video', 0.014774175360798836), ('think', 0.013109712861478329), ('system', 0.01248553954064846), ('content', 0.0110291363671422), ('ne

In [10]:
# Print the frequent words removed by rm_top. Do we want these in the model?
model.removed_top_words

['people', 'like', 'one', 'us', 'would']

## Visualise the topic model
The topic model can be visualised using the [pyLDAvis](https://pyldavis.readthedocs.io/en/latest/index.html) Python library, which is adapted from the R package developed by Carson Sievert and Kenny Shirley (see link to the original paper below).

The left side of the pyLDAvis chart shows us the distribution of the topics in our model, and how closely they are related. The right side of the chart shows us the most relevant terms for the selected topic. Decreasing the value of lambda using the slider on the top right lets you re-rank the words displayed to be more specific to your topic of interest. This can make it easier to see what the topic is describing. Hovering over a word will display the other topics the word appears in on the left side of the chart. You can watch a video [here](https://www.youtube.com/watch?v=IksL96ls4o0) that explains how to interpret the LDAvis chart in much more detail.

Read more:
[Sievert, C., & Shirley, K. E. (2014). LDAvis: A method for visualizing and interpreting topics. Proceedings of the Workshop on Interactive Language Learning, Visualization, and Interfaces (pp. 63-70). Baltimore: Association for Computational Linguistics.](http://nlp.stanford.edu/events/illvi2014/papers/sievert-illvi2014.pdf)

**Important note:** the pyLDAvis topic numbers will be off by one from the results in our model summary (i.e., Topic 1 in pyLDAvis will be Topic 0 from the tomotopy output above). 

In [11]:
# Adapted from https://github.com/bab2min/tomotopy/blob/main/examples/lda_visualization.py

topic_term_dists = np.stack([model.get_topic_word_dist(k) for k in range(model.k)])
doc_topic_dists = np.stack([doc.get_topic_dist() for doc in model.docs])
doc_topic_dists /= doc_topic_dists.sum(axis=1, keepdims=True)
doc_lengths = np.array([len(doc.words) for doc in model.docs])
vocab = list(model.used_vocabs)
term_frequency = model.used_vocab_freq

prepared_data = pyLDAvis.prepare(
    topic_term_dists, 
    doc_topic_dists, 
    doc_lengths, 
    vocab, 
    term_frequency,
    sort_topics=False # IMPORTANT: otherwise the topic_ids between pyLDAvis and tomotopy are not matching!
)
pyLDAvis.save_html(prepared_data, 'ldavis.html')

C:\Users\aha199\AppData\Roaming\Python\Python38\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


In [12]:
# View the html visualisation
# You can also open the html file in a browser
warnings.filterwarnings("ignore", category=DeprecationWarning) 

IFrame(src='ldavis.html', width=900, height=900)

## Examine top documents for a given topic
The following code to display the top documents is adapated from [***Topic Modeling - With Tomotopy***](https://melaniewalsh.github.io/Intro-Cultural-Analytics/05-Text-Analysis/09-Topic-Modeling-Without-Mallet.html) from the book *Introduction to Cultural Analytics & Python* by Melanie Walsh (2021).

In [13]:
topic_distributions = [list(doc.get_topic_dist()) for doc in model.docs]

topic_indiv_150_words = []
for topic_number in range(0, num_topics):
    topic_words_150 = ' '.join(word for word, prob in model.get_topic_words(topic_id=topic_number, top_n=150))
    topic_indiv_150_words.append(topic_words_150.split())

In [14]:
warnings.filterwarnings("ignore", category=DeprecationWarning) 

def make_md(string):
    display(Markdown(str(string)))

def get_top_docs(docs, sources, topic_indiv_150_words, topic_distributions, topic_index, n):
    
    sorted_data = sorted([(_distribution[topic_index], _document, _sources) 
                          for _distribution, _document, _sources
                          in zip(topic_distributions, docs, sources)], reverse=True)
    
    top_25 = ", ".join(topic_indiv_150_words[topic_index][:25])
    make_md\
    (f"### Topic {topic_index}\n\
    \n{top_25} ...\
    \n\n**Note**: words from the top 150 topic words are shown in bold in the text\n\n---")
    
    for probability, doc, source in sorted_data[:n]:
        # Make topic words bolded
        for word in topic_indiv_150_words[topic_index]:
            doc = doc.lower()
            if word in doc:
                doc = re.sub(f"\\b{word}\\b", f"**{word}**", doc)
                #doc = re.sub(f"\\b{word}\\b", f"**{word}**", doc, re.IGNORECASE)
        
        make_md(f'  \n**Topic Probability**: {probability}  \n**Source**: {source}  \n**Document**: {doc}  \n\n---')
    
    return

In [15]:
# Choose the topic number to explore
topic_no = 7

# Set the number of 'top docs' to display
num_top_docs = 5

In [16]:
get_top_docs(documents_list, 
             source_list,
             topic_indiv_150_words, 
             topic_distributions, 
             topic_index = topic_no, 
             n = num_top_docs)

### Topic 7
    
even, work, person, may, feel, learn, often, experience, someone, mind, much, many, language, two, learning, always, question, good, share, understanding, others, talk, found, truth, lives ...    

**Note**: words from the top 150 topic words are shown in bold in the text

---

  
**Topic Probability**: 0.4811021387577057  
**Source**: 2018-07-26-hugo_mercier_how_can_you_change_someone_s_mind.txt  
**Document**: three people are at a dinner party.
paul, who’s married, is looking at linda.
meanwhile, linda is looking at john, 
who’s not married.
is **someone** who’s married looking 
at **someone** who’s not married?
take a moment to think about it.

most people **answer** that there’s 
not enough **information** to tell.
and most people are wrong.
linda **must** be either married 
or not married—there are no other options.
so in either scenario, **someone** married 
is looking at **someone** who’s not married.
when presented with the explanation, 
most people change their minds
and accept the correct **answer**,
**despite** being very confident 
in their first responses.

now let’s look at **another** case.
a 2005 **study** by brendan nyhan 
and jason reifler
examined american attitudes regarding 
the justifications for the iraq war.
**researchers** presented participants 
with a news article
that showed no weapons 
of mass destruction had been **found**.
**yet** **many** participants not only continued 
to believe that wmds had been **found**,
but they **even** became more convinced 
of their original views.

so why do arguments change people’s minds 
in some **cases** and backfire in **others**?
arguments are more convincing when they 
rest on a **good** knowledge of the audience,
taking into account 
what the audience believes,
who they trust,
and what they value.

mathematical and logical arguments 
like the dinner party brainteaser **work**
because **even** when people 
reach **different** conclusions,
they’re starting from 
the same set of **shared** beliefs.
in 1931, a young, unknown mathematician 
named kurt gödel presented a proof
that a logically complete
system of mathematics was impossible.
**despite** upending decades of
**work** by brilliant mathematicians
like bertrand russell and david hilbert,
the proof was accepted
because it relied on axioms that 
everyone in the field **already** agreed on.

of course, **many** disagreements involve 
**different** beliefs
that can’t **simply** be reconciled
through logic.
when these beliefs involve 
**outside** **information**,
the issue **often** **comes** down to
what sources and authorities people trust.
one **study** **asked** people to estimate 
several statistics
**related** to the scope of climate change.
participants were **asked** **questions**,
such as “how **many** of the years 
between 1995 and 2006
were one of the hottest 12 years 
since 1850?”
after providing their **answers**,
they were presented with data from the
intergovernmental panel on climate change,
in this case showing that the **answer** 
was 11 of the 12 years.
being provided with these reliable 
statistics from a trusted official source
made people more **likely** to accept 
the **reality** that the earth is warming.

finally, for disagreements that 
can’t be definitively settled
with statistics or **evidence**,
making a convincing argument
**may** depend on engaging 
the audience’s values.
for **example**, **researchers** have conducted 
a number of **studies**
where they’ve **asked** people 
of **different** political backgrounds
to rank their values.
liberals in these **studies**, 
on average, rank fairness—
here **meaning** whether everyone is treated 
in the same way—above loyalty.
in later **studies**, **researchers** attempted 
to convince liberals
to support military spending with 
a variety of arguments.
arguments **based** on fairness—
like that the military provides employment
and education to people 
from disadvantaged backgrounds—
were more convincing than arguments 
**based** on loyalty—
such as that the military 
unifies a nation.

these three elements—
beliefs, trusted sources, and values—
**may** seem like a simple formula 
for finding agreement and consensus.
the problem is that our initial 
inclination is to think of arguments
that rely on our own beliefs, 
trusted sources, and values.
and **even** when we don’t,
it can be challenging to correctly 
identify what’s held dear
by people who don’t **already** agree with us.
the **best** way to find out
is **simply** to **talk** to them.
in the course of discussion,
you’ll be exposed to counter-arguments
and rebuttals.
these can **help** you make your own 
arguments and reasoning more convincing
and **sometimes**, you **may** **even** **end** up 
being the one changing your **mind**.  

---

  
**Topic Probability**: 0.44686639308929443  
**Source**: 2018-10-03-dolly_chugh_how_to_let_go_of_being_a_good_person_and_become_a_better_person.txt  
**Document**: so a **friend** of mine was riding
in a taxi to the airport the other day,
and on the way, she was chatting
with the taxi driver,
and he said to her, with total sincerity,
"i can tell you are a really **good** **person**."
and when she **told** me this story later,
she said she couldn't believe
how **good** it made her **feel**,
that it meant a lot to her.
now that **may** seem
like a strong reaction from my **friend**
to the **words** of a total stranger,
but she's not alone.

i'm a **social** scientist.
i **study** the psychology of **good** people,
and **research** in my field says
**many** of us care **deeply**
about **feeling** like a **good** **person**
and being seen as a **good** **person**.
now, your definition of "**good** **person**"
and your definition of "**good** **person**"
and maybe the taxi driver's
definition of "**good** **person**" —
we **may** not all have the same definition,
but within whatever our definition is,
that moral **identity**
is important to **many** of us.

now, if somebody challenges it,
like they **question** us for a joke we tell,
or maybe we say
our workforce is homogenous,
or a slippery business expense,
we go into red-zone defensiveness
a lot of the time.
i mean, **sometimes** we call out
all the ways in which we **help**
people from marginalized **groups**,
or we donate to charity,
or the hours we volunteer to nonprofits.
we **work** to protect
that **good** **person** **identity**.
it's important to **many** of us.

but what if i **told** you this?
what if i **told** you that our attachment
to being **good** people
is getting in the way
of us being better people?
what if i **told** you that our definition
of "**good** **person**" is so narrow,
it's scientifically impossible to **meet**?
and what if i **told** you
the path to being better people
just begins with letting go
of being a **good** **person**?

now, let me tell you a little bit
about the **research**
about how the **human** **mind** works
to **explain**.
the brain relies on shortcuts
to do a lot of its **work**.
that means a lot of the time,
your mental processes are taking place
**outside** of your awareness,
like in low-battery, low-power mode
in the back of your **mind**.
that's, in **fact**, the premise
of bounded rationality.
bounded rationality is
the nobel prize-winning idea
that the **human** **mind**
has limited storage resources,
limited processing power,
and as a result, it relies on shortcuts
to do a lot of its **work**.
so for **example**,
some scientists estimate
that in any **given** moment ...
better, better click, right? there we go.

(laughter)

at any **given** moment,
11 million pieces of **information**
are coming into your **mind**.
eleven million.
and only 40 of them
are being processed consciously.
so 11 million, 40.

i mean, has this **ever** happened to you?
have you **ever** had
a really busy day at **work**,
and you drive home,
and when you get in the door,
you realize you don't
**even** remember the drive home,
like whether you had
green lights or red lights.
you don't **even** remember.
you were on autopilot.
or have you **ever** opened the fridge,
looked for the butter,
swore there is no butter,
and then realized the butter
was right in front of you the whole time?
these are the kinds of "whoops" moments
that make us giggle,
and this is what happens in a brain
that can handle 11 million
pieces of **information** coming in
with only 40 being processed consciously.
that's the bounded part
of bounded rationality.

this **work** on bounded rationality
is what's inspired **work** i've done
with my collaborators
max bazerman and mahzarin banaji,
on what we call bounded ethicality.
so it's the same premise
as bounded rationality,
that we have a **human** **mind**
that is bounded in some sort of way
and relying on shortcuts,
and that those shortcuts
can **sometimes** lead us astray.
with bounded rationality,
**perhaps** it affects the cereal
we buy in the grocery store,
or the product we launch in the boardroom.
with bounded ethicality, the **human** **mind**,
the same **human** **mind**,
is making decisions,
and here, it's about who to hire next,
or what joke to tell
or that slippery business **decision**.

so let me give you an **example**
of bounded ethicality at **work**.
unconscious bias is one place
where we see the effects
of bounded ethicality.
so unconscious bias refers
to associations we have in our **mind**,
the shortcuts your brain is using
to organize **information**,
very **likely** **outside** of your awareness,
not necessarily lining up
with your conscious beliefs.
**researchers** nosek, banaji and greenwald
have looked at data
from millions of people,
and what they've **found** is, for **example**,
most white americans
can more quickly and easily
associate white people and **good** things
than black people and **good** things,
and most men and women
can more quickly and easily associate
men and **science** than women and **science**.
and these associations
don't necessarily line up
with what people consciously think.
they **may** have
very egalitarian views, in **fact**.
so **sometimes**, that 11 million
and that 40 just don't line up.

and here's **another** **example**:
conflicts of interest.
so we tend to underestimate
how **much** a small gift —
imagine a ballpoint pen or dinner —
how **much** that small gift
can affect our **decision** making.
we don't realize that our **mind**
is unconsciously lining up **evidence**
to support the point of view
of the gift-giver,
no **matter** how hard we're consciously
trying to be objective and professional.
we **also** see bounded ethicality —
**despite** our attachment
to being **good** people,
we **still** make **mistakes**,
and we make **mistakes**
that **sometimes** hurt other people,
that **sometimes** promote injustice,
**despite** our **best** attempts,
and we **explain** **away** our **mistakes**
rather than **learning** from them.
like, for **example**,
when i got an email
from a female student in my class
saying that a reading i had assigned,
a reading i had been assigning for years,
was sexist.
or when i confused
**two** students in my class
of the same race —
look nothing alike —
when i confused them for each other
more than once, in front of everybody.

these kinds of **mistakes** send us, send me,
into red-zone defensiveness.
they leave us fighting
for that **good** **person** **identity**.
but the latest **work** that i've been doing
on bounded ethicality with mary kern
says that we're not
only prone to **mistakes** —
that tendency **towards** **mistakes** depends
on how close we are to that red zone.
so most of the time, nobody's challenging
our **good** **person** **identity**,
and so we're not **thinking** too **much**
about the ethical implications
of our decisions,
and our model shows
that we're then spiraling
**towards** **less** and **less**
ethical **behavior** most of the time.

on the other hand, somebody
**might** challenge our **identity**,
or, upon reflection,
we **may** be challenging it ourselves.
so the ethical implications
of our decisions become really salient,
and in those **cases**, we spiral **towards**
more and more **good** **person** **behavior**,
or, to be more precise,
**towards** more and more **behavior**
that **makes** us **feel** like a **good** **person**,
which isn't **always** the same, of course.
the idea with bounded ethicality
is that we are **perhaps** overestimating
the importance our inner compass
is playing in our ethical decisions.
we **perhaps** are overestimating
how **much** our **self**-interest
is driving our decisions,
and **perhaps** we don't realize
how **much** our **self**-view as a **good** **person**
is affecting our **behavior**,
that in **fact**, we're working so hard
to protect that **good** **person** **identity**,
to keep out of that red zone,
that we're not actually giving ourselves
space to **learn** from our **mistakes**
and actually be better people.

it's **perhaps** because
we expect it to be easy.
we have this definition
of **good** **person** that's either-or.
either you are a **good** **person**
or you're not.
either you have integrity or you don't.
either you are a racist or a sexist
or a homophobe or you're not.
and in this either-or definition,
there's no room to grow.
and by the way,
this is not what we do
in most parts of our **lives**.
life, if you needed to **learn** accounting,
you would take an accounting class,
or if you become a parent,
we pick up a **book** and we read about it.
we **talk** to experts,
we **learn** from our **mistakes**,
we update our knowledge,
we just keep getting better.
but when it **comes** to being a **good** **person**,
we think it's something
we're just supposed to know,
we're just supposed to do,
without the benefit of effort or growth.

so what i've been **thinking** about
is what if we were to just forget
about being **good** people,
just let it go,
and instead, set a higher standard,
a higher standard
of being a **good**-ish **person**?
a **good**-ish **person**
absolutely **still** **makes** **mistakes**.
as a **good**-ish **person**,
i'm making them all the time.
but as a **good**-ish **person**,
i'm trying to **learn** from them, own them.
i expect them and i go after them.
i **understand** there are costs
to these **mistakes**.
when it **comes** to issues like ethics
and bias and diversity and inclusion,
there are real costs to real people,
and i accept that.
as a **good**-ish **person**, in **fact**,
i become better
at noticing my own **mistakes**.
i don't wait for people to point them out.
i practice finding them,
and as a result ...
sure, **sometimes** it can be embarrassing,
it can be uncomfortable.
we put ourselves
in a vulnerable place, **sometimes**.
but through all that vulnerability,
just like in everything else
we've tried to **ever** get better at,
we see progress.
we see growth.
we allow ourselves to get better.

why wouldn't we give ourselves that?
in every other part of our **lives**,
we give ourselves room to grow —
except in this one, where it matters most.

thank you.

(applause)  

---

  
**Topic Probability**: 0.4324553608894348  
**Source**: 2018-08-28-elizabeth_cox_what_is_imposter_syndrome_and_how_can_you_combat_it.txt  
**Document**: **even** after writing eleven books 
and winning several prestigious awards,
maya angelou couldn’t escape 
the nagging doubt
that she hadn’t really earned 
her accomplishments.
albert einstein **experienced** 
something similar:
he described himself 
as an “involuntary swindler”
whose **work** didn’t deserve 
as **much** **attention** as it had received.

accomplishments at the level 
of angelou’s or einstein’s are rare,
but their **feeling** of fraudulence 
is extremely **common**.
why can’t so **many** of us shake **feelings**
that we haven’t earned 
our accomplishments,
or that our **ideas** and skills 
aren’t worthy of **others**’ **attention**?

psychologist pauline rose clance 
was the first to **study**
this unwarranted **sense** of insecurity.
in her **work** as a therapist,
she noticed **many** of her undergraduate 
patients **shared** a concern:
**though** they had high grades,
they didn’t believe they deserved 
their spots at the university.
some **even** believed their acceptance 
had been an admissions error.
while clance knew these fears 
were unfounded,
she could **also** remember **feeling**
the exact same way in graduate school.
she and her patients **experienced**
something that goes by a number of names—
imposter phenomenon,
imposter **experience**,
and imposter syndrome.

together with colleague suzanne imes,
clance first studied imposterism
in female college students and faculty.
their **work** established pervasive
**feelings** of fraudulence in this **group**.
since that first **study**,
the same thing has been established 
across gender,
race,
age,
and a huge range of occupations,
**though** it **may** be more prevalent 
and disproportionately affect
the **experiences** of underrepresented 
or disadvantaged **groups**.
to call it a syndrome 
is to downplay how universal it is.
it's not a disease or an abnormality,
and it isn’t necessarily 
tied to depression,
anxiety,
or **self**-esteem.

where do these **feelings** 
of fraudulence come from?
people who are highly skilled 
or accomplished
tend to think **others** are just as skilled.
this can spiral into **feelings** 
that they don’t deserve accolades
and opportunities over other people.
and as angelou and einstein **experienced**,
there’s **often** no threshold 
of accomplishment
that puts these **feelings** to rest.
**feelings** of imposterism aren’t restricted 
to highly skilled individuals, either.
everyone is susceptible to a phenomenon 
known as pluralistic ignorance,
where we each doubt ourselves privately,
but believe we’re alone 
in **thinking** that way
because no one else voices their doubts.
since it’s tough to really know 
how hard our peers **work**,
how **difficult** they find **certain** tasks,
or how **much** they doubt themselves,
there’s no easy way to dismiss **feelings** 
that we’re **less** capable
than the people around us.
intense **feelings** of imposterism
can prevent people 
from sharing their great **ideas**
or applying for jobs 
and programs where they’d excel.

at least so **far**,
the most surefire way 
to combat imposter syndrome
is to **talk** about it.
**many** people suffering 
from imposter syndrome
are afraid that if they **ask** 
about their performance,
their fears will be confirmed.
and **even** when 
they receive positive feedback,
it **often** fails to ease 
**feelings** of fraudulence.
but on the other hand,
hearing that an advisor or mentor has
**experienced** **feelings** of imposterism
can **help** relieve those **feelings**.
the same goes for peers.
**even** **simply** finding out there’s a term 
for these **feelings**
can be an incredible relief.
once you’re aware of the phenomenon,
you can combat your own imposter syndrome 

by collecting 
and revisiting positive feedback.
one scientist who kept blaming herself 
for problems in her lab
started to document the causes 
every time something went wrong.
eventually, she realized most 
of the problems
came from equipment failure,
and came to recognize her own competence.

we **may** never be able 
to banish these **feelings** entirely,
but we can have open **conversations** 
about academic or professional challenges.
with increasing awareness 
of how **common** these **experiences** are,
**perhaps** we can **feel** freer to be frank 
about our **feelings**
and build confidence 
in some simple truths:
you have talent,
you are capable,
and you belong.  

---

  
**Topic Probability**: 0.42521584033966064  
**Source**: 2018-11-26-hayley_levitt_who_decides_what_art_means.txt  
**Document**: imagine you and a **friend** are 
strolling through an art exhibit
and a striking painting catches your eye.
the vibrant red appears to you 
as a symbol of love,
but your **friend** is convinced 
it's a symbol of war.
and where you see stars in a romantic sky,
your **friend** interprets global 
warming-inducing pollutants.
to settle the **debate**, you turn to the
internet, where you read
that the painting is a replica of 
the artist's first-grade art project:
red was her favorite color 
and the silver dots are fairies.

you now know the exact intentions 
that led to the creation of this **work**.
are you wrong to have enjoyed it 
as something the artist didn’t intend?
do you enjoy it **less** now 
that you know the **truth**?
just how **much** should 
the artist's intention
affect your interpretation 
of the painting?
it's a **question** that's been tossed around
by philosophers and art critics for 
decades, with no consensus in sight.

in the mid-20th century,
literary critic w.k. wimsatt and 
philosopher monroe beardsley
argued that artistic 
intention was irrelevant.
they called this the intentional fallacy:
the belief that valuing an artist's 
intentions was misguided.
their argument was twofold:
first, the artists we **study** are 
no longer living,
never recorded their intentions,
or are **simply** unavailable to **answer** 
**questions** about their **work**.
second, **even** if there were a bounty 
of relevant **information**,
wimsatt and beardsley believed
it would distract us from the 
qualities of the **work** itself.
they compared art to a dessert:
when you taste a pudding,
the chef's intentions don't affect whether
you enjoy its flavor or texture.
all that matters, they said, 
is that the pudding "works."

of course, what "works" for one **person** 
**might** not "**work**" for **another**.
and since **different** interpretations 
appeal to **different** people,
the silver dots in our painting could be 
reasonably interpreted as fairies,
stars, or pollutants.
by wimsatt and beardsley's logic, the
artist's interpretation of her own **work**
would just be one **among** **many** equally
acceptable possibilities.

if you find this problematic,
you **might** be more in line with steven 
knapp and walter benn michaels,
**two** literary theorists who rejected the
intentional fallacy.
they argued that an artist's 
intended **meaning**
was not just one **possible** interpretation,
but the only **possible** interpretation.
for **example**, suppose you're 
walking along a beach
and come across a **series** of marks in the 
sand that spell out a verse of poetry.
knapp and michaels believed the 
poem would lose all **meaning**
if you discovered these marks were not 
the **work** of a **human** being,
but an odd coincidence 
produced by the waves.
they believed an intentional creator
is what **makes** the poem subject to 
**understanding** at all.

other thinkers advocate for 
a middle ground,
suggesting that intention is just one 
piece in a larger puzzle.
contemporary philosopher noel carroll 
took this stance,
arguing that an artist's intentions are 
relevant to their audience
the same way a speaker's intentions
are relevant to the **person** they’re 
engaging in **conversation**.
to **understand** how intentions **function** 
in **conversation**,
carroll said to imagine **someone** holding
a cigarette and **asking** for a match.
you **respond** by handing them a lighter,
gathering that their motivation is to 
light their cigarette.
the **words** they used to **ask** the **question** 
are important,
but the intentions behind the **question** 
dictate your **understanding** and ultimately,
your **response**.

so which **end** of this spectrum 
do you lean **towards**?
do you, like wimsatt and beardsley, 
believe that when it **comes** to art,
the proof should be in the pudding?
or do you think that an artist's plans 
and motivations for their **work**
affect its **meaning**?
artistic interpretation is a complex web
that will probably never offer 
a definitive **answer**.  

---

  
**Topic Probability**: 0.3784974217414856  
**Source**: 2018-04-10-zachary_r_wood_why_it_s_worth_listening_to_people_we_disagree_with.txt  
**Document**: in 1994,
charles murray and richard herrnstein
coauthored "the bell curve,"
an extremely controversial **book**
which claims that on average,
some races are smarter
and more **likely** to succeed than **others**.
murray and herrnstein **also** suggest
that a lack of critical intelligence
explains the prominence of violent crime
in poor african-american communities.
but charles murray and richard herrnstein
are not the only people who think this.

in 2012,
a writer, journalist and political
commentator named john derbyshire
wrote an article that was supposed to be
a non-black version of the **talk**
that **many** black parents **feel**
they have to give their kids today:
advice on how to stay safe.
in it, he offered suggestions such as:
"do not attend events
**likely** to draw a lot of blacks,"
"stay out of heavily black neighborhoods"
and "do not act the **good** samaritan
to blacks in distress."
and **yet**, in 2016,
i invited john derbyshire
as well as charles murray
to speak at my school,
**knowing** full well that i would
be giving them a platform and **attention**
for **ideas** that i despised and rejected.
but this is just a further evolution
of a journey of uncomfortable **learning**
throughout my life.

when i was 10 years old, my mother
was diagnosed with schizophrenia,
a mental illness characterized
by mood swings and paranoid delusions.
throughout my life, my mother's rage
would turn our small house
into a minefield.
**yet**, **though** i feared
her rage on a daily basis,
i **also** learned so **much** from her.
our relationship was complicated
and challenging,
and at the age of 14, it was decided
that i needed to live apart from her.
but over the years,
i've come to appreciate
some of the important lessons
my mother taught me about life.
she was the first **person** who spoke to me
about **learning** from the other side.
and she, like me, was born and raised
in a family of committed
liberal democrats.
**yet**, she encouraged me to see the world
and the issues our world faces
as complex, controversial
and **ever**-changing.

one day, i came across
the phrase "affirmative action"
in a **book** i was reading.
and when i **asked** her what the term meant,
she **spent** what felt like an hour
giving me a thorough
and thoughtful explanation
that would make **sense** to a small child.
she **even** made the topic sound
at least as interesting
as any of my professors have.
she explained the **many** reasons
why people of various political views
challenge and support affirmative action,
stressing that, while she strongly
supported it herself,
it was important for me to view the issue
as a controversial one
with a long history,
a questionable future
and a host of complicating factors.
while affirmative action can increase
the presence of minorities
at elite educational institutions,
she felt that it could **also** disadvantage
hardworking people of **different** races
from more affluent backgrounds.
my mom wanted me to **understand**
that i should never
just write off opinions
that i disagreed with or disliked,
because there was **always** something
to **learn** from the perspectives of **others**,
**even** when doing so **might** be **difficult**.

but life at home with my mom
was not the only aspect of my journey
that has been formative and uncomfortable.
in fourth grade, she decided
that i should attend a private school
in order to receive
the **best** education **possible**.
as a black student attending
predominantly white private schools,
i've encountered attitudes and behaviors
that reflected racial stereotypes.
several of my friends' parents
assumed within minutes of meeting me
that my **best** skill was playing basketball.
and it really upset me to think
that my race made it harder for them
to see me as a student who loved
reading, writing and speaking.
**experiences** like this motivated me
to **work** tirelessly
to disprove what i knew
people had assumed.
my mother **even** said that,
in order to put my **best** foot forward,
i had to be patient, alert
and excruciatingly well-mannered.
to prove that i belonged,
i had to show poise and confidence,
the ability to speak well
and **listen** closely.
only then would my peers see
that i deserved to be there
as **much** as they did.

**despite** this racial stereotyping
and the discomfort i **often** felt,
the **learning** i gained from other aspects
of being at an elite private school
were incredibly valuable.
i was encouraged by my teachers
to explore my curiosity,
to challenge myself in new ways
and to deepen my **understanding**
of subjects that fascinated me the most.
and going to college was the next step.
i was excited to take my intellectual
drive and interest in the world of **ideas**
to the next level.
i was eager to engage in lively **debate**
with peers and professors
and with **outside** speakers;
to **listen**, to **learn** and gain
a deeper **understanding** of myself
and of **others**.
while i was fortunate to **meet**
peers and professors
who were interested
in doing the same thing,
my desire to engage with **difficult** **ideas**
was **also** met with resistance.

to prepare myself to engage
with controversy in the real world,
i joined a **group** that brought
controversial speakers to campus.
but **many** people fiercely
opposed this **group**,
and i received significant pushback
from students, faculty
and my administration.
for **many**, it was **difficult** to see
how bringing controversial
speakers to campus could be valuable,
when they caused harm.
and it was disappointing to me
facing **personal** attacks,
having my administration cancel speakers
and hearing my intentions
distorted by those around me.
my **work** **also** hurt the **feelings** of **many**,
and i understood that.
of course, no one likes being offended,
and i certainly don't like hearing
controversial speakers
argue that feminism has become
a war against men
or that blacks have lower iqs than whites.
i **also** **understand**
that some people have **experienced**
traumatic **experiences** in their **lives**.
and for some, **listening** to offensive views
can be like reliving the very traumas
that they've worked so hard to overcome.
**many** argue that by giving
these people a platform,
you're doing more harm than **good**,
and i'm reminded of this every time
i **listen** to these points of view
and **feel** my stomach turn.

**yet**, tuning out opposing viewpoints
doesn't make them go **away**,
because millions of people
agree with them.
in order to **understand**
the **potential** of society
to progress forward,
we need to **understand** the counterforces.
by engaging with controversial
and offensive **ideas**,
i believe that we can find **common** ground,
if not with the speakers themselves,
then with the audiences
they **may** attract or indoctrinate.
through engaging, i believe
that we **may** reach a better **understanding**,
a deeper **understanding**,
of our own beliefs
and preserve the ability
to solve problems,
which we can't do
if we don't **talk** to each other
and make an effort to be **good** listeners.

but soon after i announced
that john derbyshire
would be speaking on campus,
student backlash erupted on **social** **media**.
the tide of resistance,
in **fact**, was so intense,
that my college president
rescinded the invitation.
i was **deeply** disappointed by this
because, as i saw it,
there would be nothing
that any of my peers or i could do
to silence **someone** who agreed with him
in the office environment
of our future employers.

i look out at what's happening
on college campuses,
and i see the anger.
and i get it.
but what i wish i could tell people
is that it's worth the discomfort,
it's worth **listening**,
and that we're stronger,
not weaker, because of it.
when i think about my **experiences**
with uncomfortable **learning**,
and i reflect upon them,
i've **found** that it's been very **difficult**
to change the values
of the intellectual community
that i've been a part of.
but i do **feel** a **sense** of hope
when i think about the individual
interactions that i've been able to have
with students who both support
the **work** that i'm doing
and who **feel** challenged by it
and who do not support it.
what i've **found** is that,
while it can be **difficult** to change
the values of a community,
we can gain a lot
from individual interactions.

while i didn't get to engage
with john derbyshire
due to my president's disinvitation,
i was able to have dinner
with charles murray before his **talk**.
i knew the **conversation**
would be **difficult**.
and i didn't expect it to be pleasant.
but it was cordial, and i did gain
a deeper **understanding** of his arguments.
i **found** that he, like me,
believed in creating a more just society.
the thing is, his **understanding**
of what justice entailed
was very **different** from my own.
the way in which he wanted
to **understand** the issue,
the way in which he wanted
to approach the issue of inequality
**also** differed from my own.
and i **found** that his **understanding**
of issues like welfare
and affirmative action
was tied and **deeply** rooted
in his **understanding** of various
libertarian and conservative beliefs,
what diminishes and increases
their presence in our society.
while he expressed
his viewpoints eloquently,
i remained thoroughly unconvinced.
but i did walk **away**
with a deeper **understanding**.

it's my belief
that to achieve progress
in the face of adversity,
we need a genuine commitment
to gaining a deeper
**understanding** of humanity.
i'd like to see a world with more leaders
who are familiar with
the depths of the views
of those they **deeply** disagree with,
so that they can **understand** the nuances
of everyone they're representing.
i see this as an ongoing process
involving constant **learning**,
and i'm confident that i'll be able
to add value down the line
if i continue building **empathy**
and **understanding**
through engaging
with unfamiliar perspectives.

thank you.

(applause)  

---

## Get a coherence score
Topic coherence scores attempt to measure how similar the words in a topic are to each other and, as a result, how easy the topic is for a human to interpret. There are a number of topic coherence metrics, some of which, you can read a little more about them [here](http://svn.aksw.org/papers/2015/WSDM_Topic_Evaluation/public.pdf) and [here](https://github.com/dice-group/Palmetto/wiki/Coherences). 

The code below uses the tomotopy presets to calculate the average coherence for the model, and the coherence per topic using the following coherence metrics:
* u_mass
* c_uci
* c_npmi
* c_v (not recommended anymore - use 'c_npmi' instead)

In [ ]:
# calculate coherence scores

for preset in ('u_mass', 'c_uci', 'c_npmi', 'c_v'):
    coh = tp.coherence.Coherence(model, coherence=preset)
    average_coherence = coh.get_score()
    coherence_per_topic = [coh.get_score(topic_id=k) for k in range(model.k)]
    print('==== Coherence : {} ===='.format(preset))
    print('Average:', average_coherence, '\nPer Topic:', coherence_per_topic)
    print()
    